# <center> Shearlab decomposition benchmarks </center>

In [1]:
# Importing julia
import julia
import matplotlib.pyplot as plt
%matplotlib inline
import pylab
from math import ceil
from numpy.fft import fft2, ifft2, fftshift, ifftshift
import numpy as np
import pyshearlab

In [2]:
def tic():
    #Homemade version of matlab tic and toc functions
    import time
    global startTime_for_tictoc
    startTime_for_tictoc = time.time()

def toc():
    import time
    if 'startTime_for_tictoc' in globals():
        print("Elapsed time is " + str(time.time() - startTime_for_tictoc) + " seconds.")
    else:
        print("Toc: start time not set")

In [3]:
# Function to load Shearlab
def load_Shearlab():
    # Importing base
    j = julia.Julia()
    j.eval('using Shearlab')
    j.eval('using PyPlot')
    j.eval('using Images')
    return j

In [35]:
# Function to load images with certain size
def load_image(name, n, m=None, gpu=0, square=0):
    if m is None:
        m = n
    command = ('Shearlab.load_image("{}", {}, {}, {}, {})'
           ''.format(name, n, m, gpu, square))
    return j.eval(command)

In [5]:
# Function to plot images
def imageplot(f, str='', sbpt=[]):
    """
        Use nearest neighbor interpolation for the display.
    """
    if sbpt != []:
        plt.subplot(sbpt[0], sbpt[1], sbpt[2])
    imgplot = plt.imshow(f, interpolation='nearest')
    imgplot.set_cmap('gray')
    pylab.axis('off')
    if str != '':
        plt.title(str)

In [6]:
# Class of shearlet system in 2D
class Shearletsystem2D:
    def __init__(self, shearlets, size, shearLevels, full, nShearlets,
                 shearletIdxs, dualFrameWeights, RMS, isComplex):
        self.shearlets = shearlets
        self.size = size
        self.shearLevels = shearLevels
        self.full = full
        self.nShearlets = nShearlets
        self.shearletIdxs = shearletIdxs
        self.dualFrameWeights = dualFrameWeights
        self.RMS = RMS
        self.isComplex = isComplex

In [7]:
# Function to generate de 2D system
def getshearletsystem2D(rows, cols, nScales, shearLevels=None,
                        full=0,
                        directionalFilter='Shearlab.filt_gen("directional_shearlet")',
                        quadratureMirrorFilter='Shearlab.filt_gen("scaling_shearlet")'):
    if shearLevels is None:
        shearLevels = [float(ceil(i / 2)) for i in range(1, nScales + 1)]
    j.eval('rows=' + str(rows))
    j.eval('cols=' + str(cols))
    j.eval('nScales=' + str(nScales))
    j.eval('shearLevels=' + str(shearLevels))
    j.eval('full=' + str(full))
    j.eval('directionalFilter=' + directionalFilter)
    j.eval('quadratureMirrorFilter=' + quadratureMirrorFilter)
    j.eval('shearletSystem=Shearlab.getshearletsystem2D(rows,cols, nScales, shearLevels, full, directionalFilter,quadratureMirrorFilter) ')
    shearlets = j.eval('shearletSystem.shearlets')
    size = j.eval('shearletSystem.size')
    shearLevels = j.eval('shearletSystem.shearLevels')
    full = j.eval('shearletSystem.full')
    nShearlets = j.eval('shearletSystem.nShearlets')
    shearletIdxs = j.eval('shearletSystem.shearletIdxs')
    dualFrameWeights = j.eval('shearletSystem.dualFrameWeights')
    RMS = j.eval('shearletSystem.RMS')
    isComplex = j.eval('shearletSystem.isComplex')
    return Shearletsystem2D(shearlets, size, shearLevels, full, nShearlets,
                            shearletIdxs, dualFrameWeights, RMS, isComplex)

In [8]:
# Shearlet Decomposition function
def sheardec2D(X, shearletSystem):
    coeffs = np.zeros(shearletSystem.shearlets.shape, dtype=np.complex_)
    Xfreq = fftshift(fft2(ifftshift(X)))
    for i in range(shearletSystem.nShearlets):
        coeffs[:, :, i] = fftshift(ifft2(ifftshift(Xfreq * np.conj(
                                   shearletSystem.shearlets[:, :, i]))))
    return coeffs.real

In [9]:
# Shearlet Recovery function
def shearrec2D(coeffs, shearletSystem):
    X = np.zeros((coeffs.shape[0], coeffs.shape[1]), dtype=np.complex_)
    for i in range(shearletSystem.nShearlets):
        X = X + fftshift(fft2(ifftshift(coeffs[:, :, i]))) * shearletSystem.shearlets[:, :, i]
    return (fftshift(ifft2(ifftshift((
            1 / shearletSystem.dualFrameWeights) * X)))).real

In [10]:
# Shearlet Decomposition adjoint function
def sheardecadjoint2D(coeffs, shearletSystem):
    X = np.zeros((coeffs.shape[0], coeffs.shape[1]), dtype=complex)
    for i in range(shearletSystem.nShearlets):
        X = X + fftshift(fft2(ifftshift(coeffs[:, :, i]))) * np.conj(shearletSystem.shearlets[:, :, i])
    return fftshift(ifft2(ifftshift(X))).real

In [11]:
# Shearlet Recovery adjoint function
def shearrecadjoint2D(X, shearletSystem):
    coeffs = np.zeros(shearletSystem.shearlets.shape, dtype=np.complex_)
    Xfreq = fftshift(fft2(ifftshift(X)))
    for i in range(shearletSystem.nShearlets):
        coeffs[:, :, i] = fftshift(ifft2(ifftshift(
            Xfreq * shearletSystem.shearlets[:, :, i])))
    return coeffs.real

## Pure julia function

In [30]:
j = load_Shearlab()

In [13]:
n = 512
m = n
gpu = 0
square = 0
name = './data_samples/lena.jpg';

In [14]:
nScales = 4;
shearLevels = [float(ceil(i/2)) for i in range(1,nScales+1)]
scalingFilter = 'Shearlab.filt_gen("scaling_shearlet")'
directionalFilter = 'Shearlab.filt_gen("directional_shearlet")'
waveletFilter = 'Shearlab.mirror(scalingFilter)'
scalingFilter2 = 'scalingFilter'
full = 0;

In [15]:
# Read Data
j.eval('n = 512;')
# The path of the image
j.eval('name = "./lena.jpg";');

In [16]:
# Pure Julia
tic()
j.eval('X = Shearlab.load_image(name, n);');
toc()

Elapsed time is 6.044372320175171 seconds.


In [36]:
# Julia/Python
tic()
data = load_image(name,n);
toc()

Elapsed time is 0.056955575942993164 seconds.


In [18]:
sizeX = data.shape[0]
sizeY = data.shape[1]
rows = sizeX
cols = sizeY
X = data;

** Shearlet System generation **

In [27]:
# Python/Julia
tic()
shearletSystem_jl = getshearletsystem2D(rows,cols,nScales,shearLevels,full,directionalFilter,scalingFilter);
toc()

Elapsed time is 9.282433986663818 seconds.


In [20]:
# pyShearlab
tic()
shearletSystem_py = pyshearlab.SLgetShearletSystem2D(0,rows, cols, nScales)
toc()

Elapsed time is 44.695961713790894 seconds.


** Coefficients computation **

In [21]:
# Pure Julia
tic()
j.eval('coeffs = Shearlab.SLsheardec2D(X,shearletSystem);');
toc()

Elapsed time is 1.771695613861084 seconds.


In [28]:
# Julia/Python
tic()
coeffs_jl = sheardec2D(X,shearletSystem_jl);
toc()

Elapsed time is 2.234919309616089 seconds.


In [23]:
# pyShearlab
tic()
coeffs_py = pyshearlab.SLsheardec2D(X, shearletSystem_py)
toc()

Elapsed time is 1.7131218910217285 seconds.


** Reconstruction **

In [24]:
# Pure Julia
tic()
j.eval('Xrec=Shearlab.SLshearrec2D(coeffs,shearletSystem);');
toc()

Elapsed time is 1.3021342754364014 seconds.


In [25]:
# Julia/Python
tic()
Xrec_jl = shearrec2D(coeffs_jl,shearletSystem_jl);
toc()

Elapsed time is 1.525679349899292 seconds.


In [29]:
# pyShearlab
tic()
Xrec_py = pyshearlab.SLshearrec2D(coeffs_py, shearletSystem_py)
toc()

Elapsed time is 1.102597951889038 seconds.
